In [23]:
import pandas as pd
from tqdm import tqdm

In [24]:
import random

def random_model(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["logit_0"] = pd.Series([random.uniform(-1, 1) for _ in range(len(df))])
    df["logit_1"] = pd.Series([random.uniform(-1, 1) for _ in range(len(df))])
    return df

In [25]:
df = pd.read_pickle("../results/predictions_finetuned_2e-5_bs32_e3_best.pkl")
# df = random_model(df)
print(len(df))
df.head()


1532682


,book_path,chapter_idx,paragraph_idx,labels,logit_0,logit_1
0,riehl_ovidhofe.json,0,1,1,-3.513684,2.484013
1,riehl_ovidhofe.json,0,2,1,-4.145504,2.869292
2,riehl_ovidhofe.json,0,3,1,-1.776233,1.367002
3,riehl_ovidhofe.json,0,4,1,-4.529003,3.121587
4,riehl_ovidhofe.json,0,5,1,-3.214940,2.548427


In [26]:
def add_predictions(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["top_n_prediction"] = pd.Series([True for _ in range(len(df))])
    for book in tqdm(df["book_path"].unique()):
        book_df = df[df["book_path"] == book]
        predicted_chapters = book_df.sort_values("logit_0", ascending=False)[
            : len(book_df[book_df["labels"] == 0])
        ]
        for row in predicted_chapters.iterrows():
            df.loc[row[0], "top_n_prediction"] = False  # chapter break is class 0
    df["local_prediction"] = pd.Series([True for _ in range(len(df))])
    for idx, row in df.iterrows():
        df.loc[idx, "local_prediction"] = row["logit_0"] < row["logit_1"]

    return df

pred_df = add_predictions(df)
pred_df.head()

100%|██████████| 1188/1188 [01:50<00:00, 10.79it/s]


,book_path,chapter_idx,paragraph_idx,labels,logit_0,logit_1,top_n_prediction,local_prediction
0,riehl_ovidhofe.json,0,1,1,-3.513684,2.484013,True,True
1,riehl_ovidhofe.json,0,2,1,-4.145504,2.869292,True,True
2,riehl_ovidhofe.json,0,3,1,-1.776233,1.367002,True,True
3,riehl_ovidhofe.json,0,4,1,-4.529003,3.121587,True,True
4,riehl_ovidhofe.json,0,5,1,-3.214940,2.548427,True,True


In [27]:
import json
from sklearn.metrics import f1_score
from nltk.metrics import segmentation

def calc_f1_score(df: pd.DataFrame, prediction_column: str) -> float:
    # class 0 (chapter break) is relevant for us
    return f1_score(
        df["labels"].replace([1, 0], value=[False, True]),
        df[prediction_column].replace([True, False], value=[False, True]),
    )


def calc_pk_wd(df: pd.DataFrame, prediction_column: str) -> tuple:
    # class 0 (no continuation) should denote chapter break, metrics assume 1
    labels = df["labels"].replace([1, 0], value=[0, 1])
    predictions = df[prediction_column].replace([True, False], value=[0, 1])
    labels = "".join(map(str, labels))
    predictions = "".join(map(str, predictions))
    # k should be half average reference segment length
    # average paragraph count per chapter is ~54
    # median is 32
    k = int(round(len(labels) / (predictions.count("1") * 2.0)))
    pk = segmentation.pk(labels, predictions, k=k)
    windowdiff = segmentation.windowdiff(labels, predictions, k=k)
    return pk, windowdiff

def calculate_metrics(df: pd.DataFrame, prediction_column: str) -> pd.DataFrame:
    results = []
    for book in tqdm(df["book_path"].unique()):
        book_df = df[df["book_path"] == book]
        f1 = calc_f1_score(book_df, prediction_column)
        pk, windowdiff = calc_pk_wd(book_df, prediction_column)
        with open(f"../corpus/{book}", "r") as f:
            genre = json.load(f)["genre"]
        results.append({"book": book, "f1": f1, "pk": pk, "wd": windowdiff, "genre": genre})
    return pd.DataFrame(results)

In [28]:
top_n_df = calculate_metrics(pred_df, "top_n_prediction")
print(top_n_df.head())

print("Micro-averaged F1: ", top_n_df["f1"].mean())
print("Micro-averaged PK: ", top_n_df["pk"].mean())
print("Micro-averaged WD: ", top_n_df["wd"].mean())

100%|██████████| 1188/1188 [01:54<00:00, 10.40it/s]

                     book        f1        pk        wd  \
0     riehl_ovidhofe.json  0.400000  0.367424  0.522727   
1   dumasalt_margot2.json  0.500000  0.332512  0.392969   
2  liebernt_knutarne.json  0.562500  0.240924  0.308345   
3   wulffen_argobast.json  0.666667  0.193755  0.266467   
4      verne_ferien2.json  0.928571  0.073394  0.073394   

                                genre  
0    Romane, Novellen und Erzählungen  
1  Historische Romane und Erzählungen  
2    Romane, Novellen und Erzählungen  
3    Romane, Novellen und Erzählungen  
4    Romane, Novellen und Erzählungen  
Micro-averaged F1:  0.5068725074474271
Micro-averaged PK:  0.27070962159394824
Micro-averaged WD:  0.33051275784987116


In [29]:
genre_grouped = top_n_df.groupby(["genre"])
f1 = genre_grouped["f1"].mean()
pk = genre_grouped["pk"].mean()
wd = genre_grouped["wd"].mean()
count = genre_grouped["book"].count()
result = pd.concat([f1, pk, wd, count], axis=1).rename(columns={"book": "count"})
result.sort_values(by="count", ascending=False)

,f1,pk,wd,count
genre,,,,
"Romane, Novellen und Erzählungen",0.490448,0.276473,0.337452,852
"Krimis, Thriller, Spionage",0.485029,0.304755,0.359000,71
Historische Romane und Erzählungen,0.484475,0.291250,0.349775,65
"Märchen, Sagen, Legenden",0.742161,0.145684,0.190550,52
Kinderbücher ab 12 Jahren,0.566361,0.263879,0.321053,42
"Humor, Satire",0.638820,0.177778,0.245307,40
Spannung und Abenteuer,0.432700,0.332765,0.388104,19
Phantastische Literatur,0.447005,0.289703,0.360368,16
Romanhafte Biographien,0.402213,0.318070,0.375424,13


In [30]:
local_df = calculate_metrics(pred_df, "local_prediction")
local_df.head()

print("Micro-averaged F1: ", local_df["f1"].mean())
print("Micro-averaged PK: ", local_df["pk"].mean())
print("Micro-averaged WD: ", local_df["wd"].mean())

100%|██████████| 1188/1188 [01:52<00:00, 10.52it/s]

Micro-averaged F1:  0.26265731936000614
Micro-averaged PK:  0.30152238619042593
Micro-averaged WD:  0.32262614257783223


In [31]:
genre_grouped = local_df.groupby(["genre"])
f1 = genre_grouped["f1"].mean()
pk = genre_grouped["pk"].mean()
wd = genre_grouped["wd"].mean()
count = genre_grouped["book"].count()
result = pd.concat([f1, pk, wd, count], axis=1).rename(columns={"book": "count"})
result.sort_values(by="count", ascending=False)

,f1,pk,wd,count
genre,,,,
"Romane, Novellen und Erzählungen",0.251633,0.301792,0.323461,852
"Krimis, Thriller, Spionage",0.238893,0.304963,0.320607,71
Historische Romane und Erzählungen,0.234289,0.310037,0.332253,65
"Märchen, Sagen, Legenden",0.446305,0.294596,0.308500,52
Kinderbücher ab 12 Jahren,0.284126,0.297233,0.319343,42
"Humor, Satire",0.393671,0.258472,0.291075,40
Spannung und Abenteuer,0.229692,0.305548,0.324941,19
Phantastische Literatur,0.215858,0.335514,0.350282,16
Romanhafte Biographien,0.153052,0.346593,0.355063,13
